# 📊 Notebook: Transform dữ liệu từ tầng Bronze lên tầng Silver

## 🎯 Mục tiêu
Transform dữ liệu products từ tầng Bronze (raw JSON) lên tầng Silver (structured data) với các bước:
1. **Extract**: Parse JSON thành tabular format
2. **Transform**: Làm sạch và chuẩn hóa dữ liệu  
3. **Load**: Lưu vào Silver layer với schema tối ưu
4. **Document**: Tạo Data Dictionary

## 📋 Quy trình 8 bước chuẩn
1. **Import & Connect DB** - Thiết lập môi trường
2. **Load dữ liệu từ Bronze** - Đọc raw data
3. **JSON Structure Analysis** - Phân tích cấu trúc JSON
4. **Parse & Flatten JSON** - Chuyển đổi sang tabular
5. **Data Quality Check** - Đánh giá chất lượng dữ liệu
6. **Data Cleaning** - Làm sạch và chuẩn hóa
7. **Load vào Silver** - Lưu với schema tối ưu
8. **Data Dictionary** - Tạo tài liệu metadata

---


## 🔧 Bước 1: Import & Connect DB

### Mục tiêu
- Import các thư viện cần thiết
- Load biến môi trường từ file `.env`
- Tạo kết nối tới MySQL database

### Kết nối Database
- `bronze_engine` → Đọc dữ liệu gốc từ schema **Bronze**
- `silver_engine` → Ghi dữ liệu đã làm sạch sang schema **Silver**

> **📌 Lưu ý**: Đây là bước khởi tạo, chưa có quyết định gì, chỉ thiết lập môi trường làm việc.


In [1]:
import os
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
DB_SILVER = os.getenv("DB_SILVER")

# Tạo kết nối tới Bronze và Silver database
bronze_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}")
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")

print("✅ Đã kết nối Bronze và Silver database thành công")
print(f"📊 Bronze DB: {DB_BRONZE}")
print(f"📊 Silver DB: {DB_SILVER}")


✅ Đã kết nối Bronze và Silver database thành công
📊 Bronze DB: winner_bronze
📊 Silver DB: winner_silver


## 📥 Bước 2: Load dữ liệu từ Bronze

### Mục tiêu
- Lấy toàn bộ bảng `products_raw` từ Bronze database
- Hiển thị thông tin cơ bản về dataset (shape, columns)
- Xem 1 record JSON mẫu để nắm cấu trúc dữ liệu

### Quy trình
1. **Load data**: Đọc từ `products_raw` table
2. **Basic info**: Hiển thị shape và columns
3. **Sample JSON**: Parse và hiển thị 1 record mẫu

> **📌 Decision Point**: Từ JSON mẫu, ta sẽ quan sát để xác định có những nhóm trường nào (định danh, thông tin sản phẩm, giá cả, danh mục...) và quyết định sẽ extract những field nào.


In [2]:
# Load dữ liệu products từ Bronze database
products_df = pd.read_sql("SELECT * FROM products_raw", bronze_engine)

# Hiển thị thông tin cơ bản về dataset
print("=== THÔNG TIN DATASET ===")
print(f"📊 Shape: {products_df.shape}")
print(f"📋 Columns: {list(products_df.columns)}")
print(f"💾 Memory usage: {products_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị 1 record JSON mẫu để nắm cấu trúc
print("\n=== JSON STRUCTURE SAMPLE ===")
sample_json = json.loads(products_df["raw_json"].iloc[0])
print(json.dumps(sample_json, indent=2, ensure_ascii=False))


=== THÔNG TIN DATASET ===
📊 Shape: (37, 4)
📋 Columns: ['shop_id', 'product_id', 'raw_json', 'extracted_at']
💾 Memory usage: 1.03 MB

=== JSON STRUCTURE SAMPLE ===
{
  "categories": [
    {
      "id": 1290005112,
      "name": "Tam Anh"
    }
  ],
  "lifecycle_days": 0,
  "note_product": "",
  "name": "Áo thu đông.W033",
  "type": "product",
  "display_id": 94,
  "has_composite": false,
  "updated_at": "2024-12-23T08:36:41",
  "keyword": null,
  "id": "3ec3103b-ee8e-484a-abf3-d542f4487c18",
  "display_id_original": null,
  "materials": null,
  "is_sell_negative": null,
  "manipulation_warehouses": [],
  "ignore_awarded_point": null,
  "creator_id": "24c8269a-19d6-40c6-807d-fd65d528163a",
  "removed": false,
  "variations": [
    {
      "barcode": null,
      "bonus_variations": [],
      "composite_products": [],
      "display_id": "WINNER-AOTHUDONG.W033DENXS",
      "fields": [
        {
          "id": "38ce4a0a-07e2-4b0b-96cc-71fc0a403206",
          "keyValue": "DEN",
          "

## 🔍 Bước 3: JSON Structure Analysis

### Mục tiêu
- Scan toàn bộ keys trong JSON để hiểu đầy đủ cấu trúc
- Phân loại các trường theo mức độ quan trọng
- Chuẩn bị cho việc lựa chọn fields để extract

### Quy trình
1. **Scan keys**: Duyệt qua 200 records đầu để lấy tất cả keys
2. **Categorize**: Phân loại keys theo business value
3. **Decision**: Quyết định extract những field nào

> **📌 Decision Point**: Từ danh sách keys, ta sẽ quyết định chọn những cột quan trọng để parse (ví dụ: product_id, name, price, category_ids...) và bỏ các cột ít giá trị (notes, creator...).

### Phân loại trường theo mức độ quan trọng:
- **🟢 High Priority**: Định danh, thông tin cơ bản, giá cả, danh mục
- **🟡 Medium Priority**: Thông tin bổ sung, metadata
- **🔴 Low Priority**: Logs, fields ít sử dụng


In [3]:
# Scan tất cả keys trong JSON để hiểu đầy đủ cấu trúc
print("=== SCANNING JSON KEYS ===")
all_keys = set()

# Scan 200 records đầu tiên để lấy tất cả keys có thể có
for raw in products_df["raw_json"].head(200):
    d = json.loads(raw)
    all_keys.update(d.keys())

print(f"📊 Tổng số keys tìm thấy: {len(all_keys)}")
print(f"📋 Danh sách keys (sorted):")
for i, key in enumerate(sorted(all_keys), 1):
    print(f"{i:2d}. {key}")

# Phân tích cấu trúc nested
print(f"\n=== NESTED STRUCTURE ANALYSIS ===")
sample_data = json.loads(products_df["raw_json"].iloc[0])

def analyze_nested_structure(data, prefix=""):
    """Phân tích cấu trúc nested objects và arrays"""
    structure = {}
    for key, value in data.items():
        if isinstance(value, dict):
            structure[f"{prefix}{key}"] = "object"
            structure.update(analyze_nested_structure(value, f"{prefix}{key}."))
        elif isinstance(value, list) and value:
            structure[f"{prefix}{key}"] = f"array[{len(value)}]"
            if value and isinstance(value[0], dict):
                structure.update(analyze_nested_structure(value[0], f"{prefix}{key}[0]."))
        else:
            structure[f"{prefix}{key}"] = type(value).__name__
    return structure

nested_structure = analyze_nested_structure(sample_data)
print(f"📊 Cấu trúc nested objects và arrays:")
for key, value_type in sorted(nested_structure.items()):
    if "." in key or "[" in key:  # Chỉ hiển thị nested structures
        print(f"  {key}: {value_type}")


=== SCANNING JSON KEYS ===
📊 Tổng số keys tìm thấy: 47
📋 Danh sách keys (sorted):
 1. bonus_products
 2. brand_id
 3. categories
 4. category_ids
 5. creator_id
 6. custom_id
 7. description
 8. display_id
 9. display_id_original
10. duration
11. has_composite
12. hidden_fb_catalog_ids
13. hide_product_name
14. id
15. ignore_awarded_point
16. image
17. inserted_at
18. is_hide
19. is_price_by_weight
20. is_published
21. is_sell_negative
22. keyword
23. lifecycle_days
24. limit_quantity_to_warn
25. manipulation_warehouses
26. material_names
27. materials
28. measure_group_id
29. name
30. not_print_product
31. note
32. note_product
33. offline_id
34. product_attributes
35. product_links
36. removed
37. shop_id
38. shop_warranty_policy_id
39. store_links
40. supplier_product_ids
41. tags
42. third_parties
43. type
44. updated_at
45. variations
46. warning_by_variation
47. warranty_period

=== NESTED STRUCTURE ANALYSIS ===
📊 Cấu trúc nested objects và arrays:
  categories[0].id: int
  categ

## Bước 4: Parse & Flatten JSON

### Mục tiêu
- Định nghĩa hàm parse_product để extract các trường đã chọn
- Flatten các trường nested như variations, categories, product_attributes
- Aggregate thông tin từ variations (min/max price, total quantity, colors, sizes)
- Chuyển đổi từ JSON sang DataFrame tabular

### Quy trình
1. **Define parsing function**: Tạo hàm parse_product với logic extract
2. **Handle nested objects**: Extract fields từ categories và product_attributes
3. **Aggregate variations**: Tính toán min/max price, total quantity, colors, sizes
4. **Apply parsing**: Áp dụng hàm cho toàn bộ dataset

### Các trường được chọn (20 fields)
**Định danh & Cơ bản**: product_id, name, custom_id, shop_id, type, description
**Thông tin sản phẩm**: brand_id, category_ids, category_name, creator_id
**Trạng thái & Metadata**: removed, is_hide, is_published, warranty_period, lifecycle_days
**Aggregated từ variations**: min_price, max_price, total_variations, total_quantity, remain_quantity
**Thời gian**: inserted_at, updated_at

> **Decision Point**: Sau cell này, ta có bảng dạng sạch hơn để kiểm tra chất lượng dữ liệu.


In [4]:
def parse_product(row):
    """
    Parse JSON product data thành tabular format
    Extract 20 trường đã chọn với xử lý nested objects và aggregation
    """
    d = json.loads(row["raw_json"])
    
    # Xử lý nested categories object
    category_name = d.get("categories", [{}])[0].get("name") if d.get("categories") else None
    
    # Xử lý variations - aggregate thông tin
    variations = d.get("variations", [])
    
    # Tính toán aggregated fields từ variations
    if variations:
        prices = [v.get("retail_price", 0) for v in variations if v.get("retail_price")]
        quantities = []
        remain_quantities = []
        
        for v in variations:
            # Lấy quantity từ variations_warehouses
            warehouses = v.get("variations_warehouses", [])
            if warehouses:
                quantities.append(warehouses[0].get("total_quantity", 0))
                remain_quantities.append(warehouses[0].get("remain_quantity", 0))
        
        min_price = min(prices) if prices else None
        max_price = max(prices) if prices else None
        total_quantity = sum(quantities) if quantities else None
        remain_quantity = sum(remain_quantities) if remain_quantities else None
        
        # Extract colors và sizes từ product_attributes
        colors = []
        sizes = []
        for attr in d.get("product_attributes", []):
            if attr.get("name") == "Màu":
                colors = attr.get("values", [])
            elif attr.get("name") == "Size":
                sizes = attr.get("values", [])
        
        colors_str = ",".join(colors) if colors else None
        sizes_str = ",".join(sizes) if sizes else None
        
    else:
        min_price = max_price = total_quantity = remain_quantity = None
        colors_str = sizes_str = None
    
    return {
        # Định danh & Cơ bản (6 fields)
        "product_id": d.get("id"),
        "name": d.get("name"),
        "custom_id": d.get("custom_id"),
        "shop_id": d.get("shop_id"),
        "type": d.get("type"),
        "description": d.get("description"),
        
        # Thông tin sản phẩm (4 fields)
        "brand_id": d.get("brand_id"),
        "category_ids": ",".join(map(str, d.get("category_ids", []))) if d.get("category_ids") else None,
        "category_name": category_name,
        "creator_id": d.get("creator_id"),
        
        # Trạng thái & Metadata (5 fields)
        "removed": d.get("removed"),
        "is_hide": d.get("is_hide"),
        "is_published": d.get("is_published"),
        "warranty_period": d.get("warranty_period"),
        "lifecycle_days": d.get("lifecycle_days"),
        
        # Aggregated từ variations (6 fields)
        "min_price": min_price,
        "max_price": max_price,
        "total_variations": len(variations),
        "total_quantity": total_quantity,
        "remain_quantity": remain_quantity,
        "colors": colors_str,
        "sizes": sizes_str,
        
        # Thời gian (2 fields)
        "inserted_at": d.get("inserted_at"),
        "updated_at": d.get("updated_at")
    }

# Áp dụng parsing function cho toàn bộ dataset
print("=== PARSING JSON DATA ===")
products_parsed = products_df.apply(parse_product, axis=1, result_type="expand")

# Hiển thị kết quả parsing
print(f"✅ Parsing completed successfully")
print(f"📊 Parsed shape: {products_parsed.shape}")
print(f"📋 Columns extracted: {len(products_parsed.columns)}")
print(f"💾 Memory usage: {products_parsed.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print(f"\n=== SAMPLE PARSED DATA ===")
print(products_parsed.head(3))


=== PARSING JSON DATA ===
✅ Parsing completed successfully
📊 Parsed shape: (37, 24)
📋 Columns extracted: 24
💾 Memory usage: 0.03 MB

=== SAMPLE PARSED DATA ===
                             product_id              name  \
0  3ec3103b-ee8e-484a-abf3-d542f4487c18  Áo thu đông.W033   
1  6baccc22-e6a9-4fde-9bd9-52c7d2e1c4d9     Áo Khoác.W032   
2  3508fe85-4a72-4965-aec6-7a908553617a  Áo thu đông.W029   

                   custom_id    shop_id     type description brand_id  \
0  WINNER - Áo thu đông.W033  230361475  product        None     None   
1     WINNER - Áo Khoác.W032  230361475  product        None     None   
2  WINNER - Áo thu đông.W029  230361475  product        None     None   

  category_ids category_name                            creator_id  ...  \
0   1290005112       Tam Anh  24c8269a-19d6-40c6-807d-fd65d528163a  ...   
1   1290028415      Lê Phong  24c8269a-19d6-40c6-807d-fd65d528163a  ...   
2   1290021501     Truong.NQ  a9911218-e34f-4852-aaa5-8e498153f3d3  ...   

 

## Bước 5: Data Quality Check

### Mục tiêu
- Tính tỷ lệ null từng cột để quyết định giữ/lược bỏ
- Check duplicate theo product_id
- In sample values cho các field quan trọng
- Đánh giá chất lượng dữ liệu tổng thể

### Quy trình
1. **Null analysis**: Tính tỷ lệ null từng cột
2. **Duplicate check**: Kiểm tra duplicate theo primary key
3. **Value analysis**: Phân tích giá trị của các trường quan trọng
4. **Quality assessment**: Đánh giá tổng thể chất lượng dữ liệu

### Decision Rules
- Nếu cột null > 70% → cân nhắc drop
- Nếu có duplicate → xác định rule xử lý (giữ bản mới nhất)
- Nếu có giá trị bất thường → quyết định chuẩn hóa mapping

### Trường quan trọng cần kiểm tra
- removed: Boolean values
- is_hide: Boolean values  
- is_published: Boolean values
- min_price, max_price: Numeric values
- total_variations: Integer values


In [5]:
# Data Quality Analysis
print("=== DATA QUALITY ANALYSIS ===")

# 1. Null ratio analysis
print("\n1. NULL RATIO ANALYSIS")
null_ratio = products_parsed.isnull().mean().sort_values(ascending=False)
print("Tỷ lệ null theo cột (sorted):")
for col, ratio in null_ratio.items():
    print(f"  {col}: {ratio:.3f} ({ratio*100:.1f}%)")

# 2. Duplicate check
print(f"\n2. DUPLICATE CHECK")
dup_count = products_parsed["product_id"].duplicated().sum()
print(f"Số lượng product_id trùng lặp: {dup_count}")

# 3. Value analysis cho các trường quan trọng
print(f"\n3. VALUE ANALYSIS")

# Boolean fields analysis
boolean_fields = ['removed', 'is_hide', 'is_published']
for field in boolean_fields:
    if field in products_parsed.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        value_counts = products_parsed[field].value_counts(dropna=False)
        print(value_counts)

# Numeric fields analysis
numeric_fields = ['min_price', 'max_price', 'total_variations', 'total_quantity', 'remain_quantity']
for field in numeric_fields:
    if field in products_parsed.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        stats = products_parsed[field].describe()
        print(f"Count: {stats['count']}")
        print(f"Mean: {stats['mean']:.2f}")
        print(f"Min: {stats['min']}")
        print(f"Max: {stats['max']}")
        print(f"Null count: {products_parsed[field].isnull().sum()}")

# String fields analysis
string_fields = ['type', 'category_name', 'colors', 'sizes']
for field in string_fields:
    if field in products_parsed.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        value_counts = products_parsed[field].value_counts(dropna=False).head(10)
        print(value_counts)

# 4. Overall quality summary
print(f"\n4. OVERALL QUALITY SUMMARY")
total_columns = len(products_parsed.columns)
high_null_columns = len(null_ratio[null_ratio > 0.7])
print(f"Total columns: {total_columns}")
print(f"Columns with >70% null: {high_null_columns}")
print(f"Duplicate products: {dup_count}")
print(f"Data quality score: {((total_columns - high_null_columns) / total_columns * 100):.1f}%")


=== DATA QUALITY ANALYSIS ===

1. NULL RATIO ANALYSIS
Tỷ lệ null theo cột (sorted):
  brand_id: 1.000 (100.0%)
  description: 1.000 (100.0%)
  is_published: 1.000 (100.0%)
  is_hide: 1.000 (100.0%)
  warranty_period: 1.000 (100.0%)
  category_name: 0.135 (13.5%)
  category_ids: 0.135 (13.5%)
  total_quantity: 0.081 (8.1%)
  remain_quantity: 0.081 (8.1%)
  type: 0.000 (0.0%)
  custom_id: 0.000 (0.0%)
  shop_id: 0.000 (0.0%)
  product_id: 0.000 (0.0%)
  name: 0.000 (0.0%)
  removed: 0.000 (0.0%)
  creator_id: 0.000 (0.0%)
  min_price: 0.000 (0.0%)
  lifecycle_days: 0.000 (0.0%)
  total_variations: 0.000 (0.0%)
  max_price: 0.000 (0.0%)
  colors: 0.000 (0.0%)
  sizes: 0.000 (0.0%)
  inserted_at: 0.000 (0.0%)
  updated_at: 0.000 (0.0%)

2. DUPLICATE CHECK
Số lượng product_id trùng lặp: 0

3. VALUE ANALYSIS

--- REMOVED ANALYSIS ---
removed
False    37
Name: count, dtype: int64

--- IS_HIDE ANALYSIS ---
is_hide
None    37
Name: count, dtype: int64

--- IS_PUBLISHED ANALYSIS ---
is_published

## Bước 6: Data Cleaning & Transformation

### Mục tiêu
- Drop các cột có tỷ lệ null cao (>70%)
- Chuẩn hóa datetime fields
- Ép kiểu numeric cho các cột số
- Chuẩn hóa string fields (name, description)
- Xử lý các giá trị bất thường

### Quy trình
1. **Drop high-null columns**: Tự động drop cột null > 70%
2. **Datetime conversion**: Chuyển đổi string sang datetime
3. **Numeric conversion**: Ép kiểu cho các cột số
4. **String normalization**: Chuẩn hóa name và description
5. **Validation**: Kiểm tra kết quả cleaning

### Decision Rules
- Drop columns với null > 70% (configurable threshold)
- Convert datetime với error handling (coerce)
- Convert numeric với error handling (coerce)
- Standardize strings: strip whitespace, title case


In [6]:
def drop_high_null_cols(df, threshold=0.7):
    """
    Xóa các cột có tỷ lệ null > threshold (mặc định 70%)
    """
    null_ratio = df.isnull().mean()
    cols_to_drop = null_ratio[null_ratio > threshold].index.tolist()
    
    print(f"Các cột bị drop (null > {threshold*100}%): {cols_to_drop}")
    
    return df.drop(columns=cols_to_drop, errors="ignore")

# Bắt đầu data cleaning
print("=== DATA CLEANING & TRANSFORMATION ===")

# 1. Drop high-null columns
print("\n1. DROPPING HIGH-NULL COLUMNS")
products_clean = drop_high_null_cols(products_parsed, threshold=0.7)
print(f"Shape after dropping: {products_clean.shape}")

# 2. Datetime conversion
print("\n2. DATETIME CONVERSION")
datetime_cols = ["inserted_at", "updated_at"]
for col in datetime_cols:
    if col in products_clean.columns:
        print(f"Converting {col} to datetime...")
        products_clean[col] = pd.to_datetime(products_clean[col], errors="coerce")

# 3. Numeric conversion
print("\n3. NUMERIC CONVERSION")
numeric_cols = [
    "shop_id", "brand_id", "warranty_period", "lifecycle_days",
    "min_price", "max_price", "total_variations", "total_quantity", "remain_quantity"
]
for col in numeric_cols:
    if col in products_clean.columns:
        print(f"Converting {col} to numeric...")
        products_clean[col] = pd.to_numeric(products_clean[col], errors="coerce")

# 4. String normalization
print("\n4. STRING NORMALIZATION")

# Name standardization
if "name" in products_clean.columns:
    print("Standardizing name field...")
    products_clean["name"] = products_clean["name"].str.strip().str.title()

# Description standardization
if "description" in products_clean.columns:
    print("Standardizing description field...")
    products_clean["description"] = products_clean["description"].str.strip()

# Custom ID standardization
if "custom_id" in products_clean.columns:
    print("Standardizing custom_id field...")
    products_clean["custom_id"] = products_clean["custom_id"].str.strip()

# 5. Validation
print("\n5. CLEANING VALIDATION")
print(f"Final shape: {products_clean.shape}")
print(f"Final columns: {len(products_clean.columns)}")
print(f"Memory usage: {products_clean.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Check data types
print(f"\nData types summary:")
print(products_clean.dtypes.value_counts())

# Sample cleaned data
print(f"\n=== SAMPLE CLEANED DATA ===")
print(products_clean.head(3))


=== DATA CLEANING & TRANSFORMATION ===

1. DROPPING HIGH-NULL COLUMNS
Các cột bị drop (null > 70.0%): ['description', 'brand_id', 'is_hide', 'is_published', 'warranty_period']
Shape after dropping: (37, 19)

2. DATETIME CONVERSION
Converting inserted_at to datetime...
Converting updated_at to datetime...

3. NUMERIC CONVERSION
Converting shop_id to numeric...
Converting lifecycle_days to numeric...
Converting min_price to numeric...
Converting max_price to numeric...
Converting total_variations to numeric...
Converting total_quantity to numeric...
Converting remain_quantity to numeric...

4. STRING NORMALIZATION
Standardizing name field...
Standardizing custom_id field...

5. CLEANING VALIDATION
Final shape: (37, 19)
Final columns: 19
Memory usage: 0.03 MB

Data types summary:
object            9
int64             5
float64           2
datetime64[ns]    2
bool              1
Name: count, dtype: int64

=== SAMPLE CLEANED DATA ===
                             product_id              name

## Bước 7: Load vào Silver Database

### Mục tiêu
- Định nghĩa explicit schema mapping cho từng cột
- Load dữ liệu đã clean vào Silver database
- Tạo bảng dim_products với schema tối ưu
- Xác nhận load thành công

### Quy trình
1. **Schema definition**: Định nghĩa dtype mapping cho từng cột
2. **Database load**: Sử dụng to_sql với explicit schema
3. **Validation**: Kiểm tra số record load thành công
4. **Schema verification**: Xác nhận schema trong database

### Schema Strategy
- VARCHAR(100): Cho các ID fields
- VARCHAR(255): Cho name fields
- VARCHAR(500): Cho description field  
- Text(): Cho các field có thể dài (colors, sizes)
- BigInteger(): Cho shop_id và các số lớn
- DateTime(): Cho timestamp fields
- Boolean(): Cho các trường boolean

> **Decision Point**: Xác nhận số record load thành công và kiểm tra schema ở Silver có đúng với kỳ vọng.


In [7]:
from sqlalchemy.types import BigInteger, Integer, Float, DateTime, Text, VARCHAR, Boolean

# Định nghĩa schema mapping cho từng cột
print("=== SCHEMA DEFINITION ===")

dtype_mapping = {}

# ID fields - VARCHAR(100)
id_fields = ["product_id", "creator_id", "brand_id"]
for field in id_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = VARCHAR(100)

# Name fields - VARCHAR(255)  
name_fields = ["name", "custom_id", "category_name", "type"]
for field in name_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = VARCHAR(255)

# Description field - VARCHAR(500)
if "description" in products_clean.columns:
    dtype_mapping["description"] = VARCHAR(500)

# Text fields - Text()
text_fields = ["category_ids", "colors", "sizes"]
for field in text_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = Text()

# Numeric fields
if "shop_id" in products_clean.columns:
    dtype_mapping["shop_id"] = BigInteger()

# Integer fields
integer_fields = ["warranty_period", "lifecycle_days", "total_variations", "total_quantity", "remain_quantity"]
for field in integer_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = Integer()

# Float fields
float_fields = ["min_price", "max_price"]
for field in float_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = Float()

# DateTime fields
datetime_fields = ["inserted_at", "updated_at"]
for field in datetime_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = DateTime()

# Boolean fields
boolean_fields = ["removed", "is_hide", "is_published"]
for field in boolean_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = Boolean()

print(f"Schema mapping defined for {len(dtype_mapping)} columns")
print("Schema mapping:")
for col, dtype in dtype_mapping.items():
    print(f"  {col}: {dtype}")

# Load vào Silver database
print(f"\n=== LOADING TO SILVER DATABASE ===")
table_name = "dim_products"

try:
    products_clean.to_sql(
        table_name,
        con=silver_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"✅ Đã load {products_clean.shape[0]} records vào Silver: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, silver_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"✅ Verification: {loaded_count} records trong database")
    print(f"✅ Schema: {len(dtype_mapping)} columns với explicit typing")
    
except Exception as e:
    print(f"❌ Error loading to Silver: {str(e)}")
    raise


=== SCHEMA DEFINITION ===
Schema mapping defined for 19 columns
Schema mapping:
  product_id: VARCHAR(100)
  creator_id: VARCHAR(100)
  name: VARCHAR(255)
  custom_id: VARCHAR(255)
  category_name: VARCHAR(255)
  type: VARCHAR(255)
  category_ids: TEXT
  colors: TEXT
  sizes: TEXT
  shop_id: BIGINT
  lifecycle_days: INTEGER
  total_variations: INTEGER
  total_quantity: INTEGER
  remain_quantity: INTEGER
  min_price: FLOAT
  max_price: FLOAT
  inserted_at: DATETIME
  updated_at: DATETIME
  removed: BOOLEAN

=== LOADING TO SILVER DATABASE ===
✅ Đã load 37 records vào Silver: dim_products
✅ Verification: 37 records trong database
✅ Schema: 19 columns với explicit typing


## Bước 8: Data Dictionary Generation

### Mục tiêu
- Tạo Data Dictionary chi tiết cho bảng dim_products
- Document metadata của từng cột (data type, null %, unique count, sample values)
- Lưu Data Dictionary vào Excel file để tham khảo
- Tạo tài liệu tham chiếu cho Gold layer design

### Quy trình
1. **Generate dictionary**: Tạo DataFrame với metadata của từng cột
2. **Add business context**: Thêm business meaning cho các trường
3. **Export to Excel**: Lưu vào file Excel để documentation
4. **Summary report**: Tạo báo cáo tổng kết

### Data Dictionary Fields
- table_name: Tên bảng
- column_name: Tên cột
- dtype: Kiểu dữ liệu (pandas)
- sql_type: Kiểu dữ liệu SQL
- null_pct: Tỷ lệ null (%)
- unique_count: Số lượng giá trị unique
- sample_values: Các giá trị mẫu
- business_meaning: Ý nghĩa business
- extraction_date: Ngày extract

> **Decision Point**: Dựa vào Data Dictionary, kiểm tra lại schema Silver có phù hợp với Business Requirement. Nếu thiếu cột quan trọng hoặc dtype chưa chuẩn, cần quay lại bước 4-6 để chỉnh sửa.


In [8]:
# Generate Data Dictionary
print("=== GENERATING DATA DICTIONARY ===")

def get_business_meaning(column_name):
    """Get business meaning for each column"""
    business_meanings = {
        # Định danh & Cơ bản
        "product_id": "Unique product identifier (primary key)",
        "name": "Product name",
        "custom_id": "Custom product ID",
        "shop_id": "Shop identifier (foreign key to dim_shops)",
        "type": "Product type",
        "description": "Product description",
        
        # Thông tin sản phẩm
        "brand_id": "Brand identifier",
        "category_ids": "Category IDs (comma-separated)",
        "category_name": "Primary category name",
        "creator_id": "Product creator ID",
        
        # Trạng thái & Metadata
        "removed": "Product removed status (true/false)",
        "is_hide": "Product hidden status (true/false)",
        "is_published": "Product published status (true/false)",
        "warranty_period": "Warranty period in days",
        "lifecycle_days": "Product lifecycle in days",
        
        # Aggregated từ variations
        "min_price": "Minimum price across all variations",
        "max_price": "Maximum price across all variations",
        "total_variations": "Total number of variations",
        "total_quantity": "Total quantity across all variations",
        "remain_quantity": "Remaining quantity across all variations",
        "colors": "Available colors (comma-separated)",
        "sizes": "Available sizes (comma-separated)",
        
        # Thời gian
        "inserted_at": "Record creation timestamp",
        "updated_at": "Record last update timestamp"
    }
    return business_meanings.get(column_name, "No business meaning defined")

# Tạo Data Dictionary
dict_data = []
for col in products_clean.columns:
    col_info = {
        "table_name": "dim_products",
        "column_name": col,
        "dtype": str(products_clean[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": products_clean[col].isnull().sum(),
        "null_pct": round(products_clean[col].isnull().mean() * 100, 2),
        "unique_count": products_clean[col].nunique(),
        "sample_values": str(products_clean[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Append Data Dictionary vào file Excel
excel_path = "Technical_Document/Dictionary.xlsx"
try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Data_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== TRANSFORMATION SUMMARY ===")
print(f"Source records: {len(products_df)}")
print(f"Target records: {len(products_clean)}")
print(f"Columns extracted: {len(products_clean.columns)}")
print(f"Columns dropped: {len(products_parsed.columns) - len(products_clean.columns)}")
print(f"Target table: Silver.dim_products")
print(f"Data Dictionary: {excel_path}")
print(f"Transformation completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


=== GENERATING DATA DICTIONARY ===
Generated Data Dictionary for 19 columns

=== DATA DICTIONARY ===
      table_name       column_name           dtype      sql_type  null_count  \
0   dim_products        product_id          object  VARCHAR(100)           0   
1   dim_products              name          object  VARCHAR(255)           0   
2   dim_products         custom_id          object  VARCHAR(255)           0   
3   dim_products           shop_id           int64        BIGINT           0   
4   dim_products              type          object  VARCHAR(255)           0   
5   dim_products      category_ids          object          TEXT           5   
6   dim_products     category_name          object  VARCHAR(255)           5   
7   dim_products        creator_id          object  VARCHAR(100)           0   
8   dim_products           removed            bool       BOOLEAN           0   
9   dim_products    lifecycle_days           int64       INTEGER           0   
10  dim_products   

Found existing data in Technical_Document/Dictionary.xlsx, appending new data...
✅ Appended 19 rows to: Technical_Document/Dictionary.xlsx

=== TRANSFORMATION SUMMARY ===
Source records: 37
Target records: 37
Columns extracted: 19
Columns dropped: 5
Target table: Silver.dim_products
Data Dictionary: Technical_Document/Dictionary.xlsx
Transformation completed: 2025-10-12 16:39:58
